In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression

In [2]:
from matplotlib.pyplot import rcParams

rcParams['figure.figsize'] = 15, 10
rcParams["font.weight"] = "bold"
rcParams["axes.labelweight"] = "bold"
rcParams["font.size"] = 12

In [8]:
df_placas = pd.read_csv('placas_solares.csv')

df_placas

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0
...,...,...,...,...,...,...
3177,2020-06-17 22:45:00,4135001,HmiyD2TTLFNqkNe,22.150570,21.480377,0.0
3178,2020-06-17 23:00:00,4135001,HmiyD2TTLFNqkNe,22.129816,21.389024,0.0
3179,2020-06-17 23:15:00,4135001,HmiyD2TTLFNqkNe,22.008275,20.709211,0.0
3180,2020-06-17 23:30:00,4135001,HmiyD2TTLFNqkNe,21.969495,20.734963,0.0


In [4]:
df_placas.isna().sum()

DATE_TIME              0
PLANT_ID               0
SOURCE_KEY             0
AMBIENT_TEMPERATURE    0
MODULE_TEMPERATURE     0
IRRADIATION            0
dtype: int64

In [5]:
df_placas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3182 entries, 0 to 3181
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DATE_TIME            3182 non-null   object 
 1   PLANT_ID             3182 non-null   int64  
 2   SOURCE_KEY           3182 non-null   object 
 3   AMBIENT_TEMPERATURE  3182 non-null   float64
 4   MODULE_TEMPERATURE   3182 non-null   float64
 5   IRRADIATION          3182 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 149.3+ KB


In [9]:
df_placas["DATE_TIME"].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
3177    False
3178    False
3179    False
3180    False
3181    False
Name: DATE_TIME, Length: 3182, dtype: bool

In [10]:
df_placas["DATE_TIME"].isna().sum()

0

In [11]:
df_placas["DATE_TIME"][df_placas["DATE_TIME"].isna()]

Series([], Name: DATE_TIME, dtype: object)

In [12]:
df_placas[4:3300]

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0
5,2020-05-15 01:15:00,4135001,HmiyD2TTLFNqkNe,24.536092,21.968571,0.0
6,2020-05-15 01:30:00,4135001,HmiyD2TTLFNqkNe,24.638674,22.352926,0.0
7,2020-05-15 01:45:00,4135001,HmiyD2TTLFNqkNe,24.873022,23.160919,0.0
8,2020-05-15 02:00:00,4135001,HmiyD2TTLFNqkNe,24.936930,23.026113,0.0
...,...,...,...,...,...,...
3177,2020-06-17 22:45:00,4135001,HmiyD2TTLFNqkNe,22.150570,21.480377,0.0
3178,2020-06-17 23:00:00,4135001,HmiyD2TTLFNqkNe,22.129816,21.389024,0.0
3179,2020-06-17 23:15:00,4135001,HmiyD2TTLFNqkNe,22.008275,20.709211,0.0
3180,2020-06-17 23:30:00,4135001,HmiyD2TTLFNqkNe,21.969495,20.734963,0.0


In [13]:
df_placas.drop(df_placas.dropna().index)

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION


In [16]:
# No hay nada que limpiar


In [17]:
df_solar_filt = df_placas.dropna().reset_index(drop=True)
df_solar_filt

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0
...,...,...,...,...,...,...
3177,2020-06-17 22:45:00,4135001,HmiyD2TTLFNqkNe,22.150570,21.480377,0.0
3178,2020-06-17 23:00:00,4135001,HmiyD2TTLFNqkNe,22.129816,21.389024,0.0
3179,2020-06-17 23:15:00,4135001,HmiyD2TTLFNqkNe,22.008275,20.709211,0.0
3180,2020-06-17 23:30:00,4135001,HmiyD2TTLFNqkNe,21.969495,20.734963,0.0


### Análisis exploratorio, tratamiento y limpieza de datos


In [18]:
df_solar_filt.describe()

,PLANT_ID,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
count,3182.0,3182.000000,3182.000000,3182.000000
mean,4135001.0,25.531606,31.091015,0.228313
std,0.0,3.354856,12.261222,0.300836
min,4135001.0,20.398505,18.140415,0.000000
25%,4135001.0,22.705182,21.090553,0.000000
50%,4135001.0,24.613814,24.618060,0.024653
75%,4135001.0,27.920532,41.307840,0.449588
max,4135001.0,35.252486,65.545714,1.221652


In [19]:
# consultar el tipo de datos
df_solar_filt.dtypes

DATE_TIME               object
PLANT_ID                 int64
SOURCE_KEY              object
AMBIENT_TEMPERATURE    float64
MODULE_TEMPERATURE     float64
IRRADIATION            float64
dtype: object

In [20]:
df_solar_filt.dtypes[df_solar_filt.dtypes == "object"]

DATE_TIME     object
SOURCE_KEY    object
dtype: object

In [21]:
# esto no hace falta
df_solar_filt['DATE_TIME'].value_counts()

DATE_TIME
2020-05-15 00:00:00    1
2020-06-06 20:45:00    1
2020-06-06 21:15:00    1
2020-06-06 21:30:00    1
2020-06-06 21:45:00    1
                      ..
2020-05-26 17:45:00    1
2020-05-26 18:00:00    1
2020-05-26 18:15:00    1
2020-05-26 18:30:00    1
2020-06-17 23:45:00    1
Name: count, Length: 3182, dtype: int64

In [23]:
# eliminar la variable date Y source_key al ser meramente informativa
columnas_drop = ['DATE_TIME', 'SOURCE_KEY']

In [24]:
df_solar_filt2 = df_solar_filt.drop(columns = columnas_drop)
print("Tamañp tablón filtrado: ", df_solar_filt.shape)
print("Tamañode la nueva tabla: ", df_solar_filt2.shape)

Tamañp tablón filtrado:  (3182, 6)
Tamañode la nueva tabla:  (3182, 4)


In [25]:
df_solar_filt2.dtypes.value_counts()

float64    3
int64      1
Name: count, dtype: int64

### Reducción de variables
Importancia de variables

In [26]:
# generar la lista de features y la variable target
target = 'MODULE_TEMPERATURE'
features = [x for x in df_solar_filt.columns if x != target]

print(target)
print(features)

MODULE_TEMPERATURE
['DATE_TIME', 'PLANT_ID', 'SOURCE_KEY', 'AMBIENT_TEMPERATURE', 'IRRADIATION']


In [27]:
df_solar_filt2[target]

0       22.857507
1       22.761668
2       22.592306
3       22.360852
4       22.165423
          ...    
3177    21.480377
3178    21.389024
3179    20.709211
3180    20.734963
3181    20.427972
Name: MODULE_TEMPERATURE, Length: 3182, dtype: float64